In [ ]:
### Function 2b  to get statistics on independant variables for teams
def get_X_predict (team, opponent):
    teams=[team, opponent]
    df_final = x_predict[x_predict["team"].isin(teams)]
    df_final[["match_played", "position", "points", "wins", "loses",
    "drawns", "goals_for", "goals_against"]] = df_final[["match_played", "position", "points", "wins", "loses",
                                                  "drawns", "goals_for", "goals_against"]].apply(lambda x: x.diff())
    df_final= df_final.iloc[1]
    df_final = pd.DataFrame(df_final)
    df_final = df_final.transpose()
    df_final = df_final.rename(columns={'team': 'opponent'})
    df_final['team'] = teams[0]
    df_final['stadium'] = stadium
    df_final= df_final.set_index(["year", "team", "opponent"])
    var_order = ['stadium','match_played', 'position', 'points', 
                 'wins', 'loses', 'drawns', 'goals_for', 'goals_against']
    X_predict = df_final.reindex(columns = var_order)          
      
    return X_predict

X_predict = get_X_predict (team_name, opponent_name)

In [ ]:
# Let's put the years considered
years = [2020, 2021, 2022]

# Let's create a list of dataframe
results = [team_matches_20, team_matches_21, team_matches_22]

# And let's apply the transformattion for each dataframe 
for i, results_frame in enumerate(results):
    results_frame['stadium'] = results_frame['stadium'].replace({'away': 0, 'home': 1})
    results_frame['year'] = years[i]
    results_frame = results_frame.reindex(columns=['year', 'team', 'opponent', 'stadium', 'result_team'])
    results[i] = results_frame

# Concaténation des dataframes
y_stats = pd.concat(results)

#### Let's combine team statistics on independant variables into a same dataframe for X train
x_stats = pd.concat([data_on_stats_20, data_on_stats_21, data_on_stats_22])


In [ ]:
#### Let's combine all training data in a single dataframe

y_stats.reset_index(drop=True) ## drop y_train index
y_stats.set_index(['year','team']) ## Set two new index
x_stats.set_index(['year','team']) ## Set two new index

base_train = pd.merge(y_stats, x_stats, how= 'inner', on = ['year', 'team'])  ## merge y_stats and x_stats for team statistics

base_train = base_train.rename(columns={'team':'opponent', 'opponent': 'team' }) ## rename columns for a second merge
base_train = pd.merge(base_train, x_stats, how= 'inner', on = ['year', 'team'])  ## merge base_train and x_stats for opponent stats

base_train = base_train.rename(columns={'opponent':'team', 'team': 'opponent' }) ## rename in as original data

In [ ]:
#### Let's create the difference variables for independant variables  
base_train_2 = base_train.assign(match_played= base_train['match_played_y']- base_train['match_played_x'],
                                position= base_train['position_y']- base_train['position_x'], 
                                points= base_train['points_y'] - base_train['points_x'],
                                wins= base_train['wins_y'] - base_train['wins_x'],
                                loses= base_train['loses_y'] - base_train['loses_x'],
                                drawns= base_train['drawns_y'] - base_train['drawns_x'],
                                goals_for= base_train['goals_for_y'] - base_train['goals_for_x'],
                                goals_against= base_train['goals_against_y'] - base_train['goals_against_x'])

#### Create new dataframe for difference variables
base_esti = base_train_2.loc[:, ["year", "team", "opponent", "result_team", "stadium", "match_played", "position", 
                                 "points", "wins", "loses", "drawns", "goals_for", "goals_against"]]

## Create the final training dataframe putting year, team and opponent variables in index
base_training= base_esti.set_index(["year", "team", "opponent"])